In [1]:
import numpy as np
#import tensorflow as tf
from glob import glob
import os
from os.path import join,exists,basename
import json
import random
import cv2 as cv
from natsort import natsorted
import tensorflow as tf
import pickle
%matplotlib inline

2023-02-20 06:22:32.273942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 06:22:32.958354: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/cv2/../../lib64:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib:/usr/local/cuda-11.7/lib64${LD_LIBRARY_PATH:+:}
2023-02-20 06:22:32.958515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvin

In [2]:
### Information about directory src and target 
dires={'raw_videos':join('dataset_raw','smarthome_raw','alphapose','CD1','raw_video'),
'bboxs':join('dataset_raw','smarthome_raw','alphapose','CD1','bbox_pkl'),
'img_ids':join('datasets_features','smarthome','alphapose','CD1','img_id')}

In [3]:
rgb_target_dires={'co':join('datasets_features','smarthome','rgb','CD1'),
'cd':join('datasets_features','smarthome','CD1')
}

In [4]:
seq_len=32
w,h=640,480
co_classnames=natsorted(os.listdir(dires['img_ids']+'/train'))

In [5]:
co_classnames

['Drink.Fromcup', 'Readbook', 'Uselaptop', 'Usetelephone']

In [6]:
## Samples Frames  Function 

def get_sampled_frames(video_dire,img_id_list,resize=False):
    sampled_frames=[]
    cap=cv.VideoCapture(video_dire)
    for frame_number in img_id_list:
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
        sucess,frame=cap.read()
        if resize:
            frame=cv.resize(frame,(w,h))
        sampled_frames.append(frame)

    sampled_frames=np.array(sampled_frames)

    return sampled_frames

In [7]:
def get_resnet152v2(resized_height=h,resized_width=w):
    ## defining the ResNet152V2 model pretrained with imagenet dataset
    feature_extractor=tf.keras.applications.ResNet152V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(resized_height,resized_width,3),
    pooling='max',
)
    ##Defining the input size for single image                    
    inputs=tf.keras.Input((resized_height,resized_width,3))
    preprocessed=tf.keras.applications.resnet_v2.preprocess_input(inputs)

    ## Give features for each image
    outputs=feature_extractor(preprocessed)
    ## defining feature extractor model
    model=tf.keras.Model(inputs,outputs,name='feature_extractor')
    return model


resnet_model=get_resnet152v2()

2023-02-20 06:22:37.061959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 06:22:37.088492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 06:22:37.088612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 06:22:37.088954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [8]:
resnet_model.summary()

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 480, 640, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 480, 640, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 480, 640, 3)      0         
 a)                                                              
                                                                 
 resnet152v2 (Functional)    (None, 2048)              58331648  
                                                                 
Total params: 58,331,648
Trainable params: 58,187,904
Non-trainable params: 143,744
_________________________________________________________________


In [9]:
features=[]
labels=[]
evals=['train','test','val']
for eval in evals:
    for idx,classname in enumerate(co_classnames):
        img_id_dires=glob(join(dires['img_ids'],eval,classname)+'/*.npy')
        for num,img_id_dire in enumerate(img_id_dires):
            filename=basename(img_id_dire)
            video_filename=filename[:-3]+'mp4'
            bbox_filename=filename[:-3]+'pickle'
            video_dire=join(dires['raw_videos'],classname,video_filename)
            bbox_dire=join(dires['bboxs'],classname,bbox_filename)

            with open(bbox_dire,'rb') as f:
                bbox_info=pickle.load(f)
            
            img_id=np.load(img_id_dire)
            img_id=[int(i.split('.')[0])for i in img_id]
            bbox_info={'{}.jpg'.format(i):bbox_info['{}.jpg'.format(i)] for i in img_id}
            bboxs=list(bbox_info.values())
            sampled_frames=get_sampled_frames(video_dire,img_id,resize=False)
           
            feature=resnet_model.predict(sampled_frames)
            features.append(feature)
            labels.append(idx)

            target_folder=join(rgb_target_dires['co'],'noncrop','resnet152','keypoints',eval,classname)

            img_folder=join(rgb_target_dires['co'],'noncrop','resnet152','imgs',eval,classname)


            if not exists(target_folder):
                os.makedirs(target_folder)

            if not exists(img_folder):
                os.makedirs(img_folder)

            target_filedire=join(target_folder,filename)
            img_filedire=join(img_folder,filename)

            np.save(target_filedire,feature)

            np.save(img_filedire,sampled_frames)

            if (num%10 ==0):
                print('{}: {}'.format(classname,num))

        

        
features=np.array(features)
labels=np.array(labels)


2023-02-20 06:22:44.507737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700


2/2 [==============================] - 5s 651ms/step
Drink.Fromcup: 0
2/2 [==============================] - 0s 252ms/step
Drink.Fromcup: 10
2/2 [==============================] - 0s 247ms/step
Drink.Fromcup: 20
2/2 [==============================] - 0s 301ms/step
Drink.Fromcup: 30
2/2 [==============================] - 0s 250ms/step
Drink.Fromcup: 40
2/2 [==============================] - 0s 251ms/step
Drink.Fromcup: 50
2/2 [==============================] - 0s 257ms/step
Drink.Fromcup: 60
2/2 [==============================] - 0s 250ms/step
Drink.Fromcup: 70
2/2 [==============================] - 0s 249ms/step
Drink.Fromcup: 80
2/2 [==============================] - 0s 248ms/step
Drink.Fromcup: 90
2/2 [==============================] - 0s 253ms/step
Drink.Fromcup: 100
2/2 [==============================] - 0s 249ms/step
Drink.Fromcup: 110
2/2 [==============================] - 0s 252ms/step
Drink.Fromcup: 120
2/2 [==============================] - 0s 250ms/step
Drink.Fromcup: 130
2/2

: 

In [ ]:
train_features, test_features, train_labels, test_labels=train_test_split(features, labels, test_size=0.25, random_state=101,stratify=labels)

In [ ]:

saved_dires_train=join(rgb_target_dires['co'],'noncrop','densenet121','features','train')
if not os.path.exists(saved_dires_train):
    os.makedirs(saved_dires_train)

saved_dires_test=join(rgb_target_dires['co'],'noncrop','densenet121','features','test')
if not os.path.exists(saved_dires_test):
    os.makedirs(saved_dires_test)

In [ ]:
np.save(join(saved_dires_train,'features.npy'),train_features)
np.save(join(saved_dires_train,'labels.npy'),train_labels)

np.save(join(saved_dires_test,'features.npy'),test_features)
np.save(join(saved_dires_test,'labels.npy'),test_labels)